In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

secret_label = "HF Hub"
secret_value = "###"
login(token=secret_value)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Installations

In [2]:
!pip install accelerate transformers wandb einops datasets peft wandb bitsandbytes trl git+https://github.com/huggingface/peft.git evaluate rouge_score loralib

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-l7ou9__c
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-l7ou9__c
  Resolved https://github.com/huggingface/peft.git to commit e72a96f7cfdb330236c1b6dfe49be398d83f089e
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 860.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.0 MB/s eta 0:00:00
  Created wheel for peft: filename=peft-0.11.2.dev0-py3-none-any.whl size=281574 s

In [3]:
import wandb
import os

os.environ['WANDB_API_KEY'] = '###'
wandb.login()

wandb: Currently logged in as: daniteebee. Use `wandb login --relogin` to force relogin


True

In [4]:
import torch
import time
import pandas as pd
import re
import numpy as np
import string
import evaluate
from string import Template

2024-07-12 12:06:00.172427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 12:06:00.172550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 12:06:00.298789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM,AutoTokenizer,BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline,logging
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import json

In [6]:
import transformers

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [7]:
# def print_number_of_trainable_model_parameters(model):
#     trainable_model_params = 0
#     all_model_params = 0
#     for _, param in model.named_parameters():
#         all_model_params += param.numel()
#         if param.requires_grad:
#             trainable_model_params += param.numel()
#     return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

# print(print_number_of_trainable_model_parameters(model))

In [8]:
with open("/kaggle/input/llm-mcq/TeleQnA_training.txt", "r") as f:
    train_data = f.read()
    
with open("/kaggle/input/llm-mcq/TeleQnA_testing1.txt", "r") as f:
    test_data = f.read()
    
test_data=json.loads(test_data)
train_data=json.loads(train_data)

In [9]:
def dict_to_frame(text):
    data = []
    for question_id in list(text.keys()):
        question_details=text[question_id]
        data.append({
           "question_id": question_id,
           "question": question_details["question"],
           "option_1": question_details["option 1"],
           "option_2": question_details["option 2"],
           "option_3": question_details.get("option 3", None),
           "option_4": question_details.get("option 4", None),
           "category": question_details["category"],
            "answer": question_details["answer"],
            "explanation": question_details["explanation"]
         })
    return pd.DataFrame(data)

# test_df=dict_to_frame(test_data)
train_df=dict_to_frame(train_data)
# test_df['question_id']=test_df['question_id'].str.replace('question ', '').astype(int)
train_df

,question_id,question,option_1,option_2,option_3,option_4,category,answer,explanation
0,question 0,What is the purpose of the Nmfaf_3daDataManage...,To configure the MFAF to map data or analytics...,To configure the MFAF to stop mapping data or ...,To supply data or analytics from the MFAF to n...,To fetch data or analytics from the MFAF based...,Standards specifications,option 2: To configure the MFAF to stop mappin...,The Nmfaf_3daDataManagement_Deconfigure servic...
1,question 4,How does a supporting UE attach to the same co...,It requests the core network node to remember ...,It uses information stored in the UE when it w...,It relies on the SIM/USIM card for information.,It performs a fresh attach procedure.,Standards specifications,option 2: It uses information stored in the UE...,A supporting UE in a shared network attaches t...
2,question 11,When can the setting of the Privacy exception ...,Never,Only during emergency services,Anytime,Only with operator permission,Standards specifications,option 3: Anytime,The setting of the Privacy exception list can ...
3,question 12,What does the NEF notify to the AF after deter...,AF Identifier,EAS address information,DNN,DNAI(s) or the updated DNAI information,Standards specifications,option 4: DNAI(s) or the updated DNAI information,"After determining the suitable DNAI(s), NEF no..."
4,question 14,"In online charging, how are chargeable events ...",By the CTF,By the OCF,By the CGF,By the CHF,Standards specifications,option 2: By the OCF,The CTF transforms each chargeable event into ...
...,...,...,...,...,...,...,...,...,...
1456,question 8111,What services does the UMTS Bearer Service con...,Radio Access Bearer Service and Core Network B...,Te/MT Local Bearer Service and External Bearer...,GERAN and UTRAN FDD/TDD,Iu and Gb Bearer Service,Standards specifications,option 1: Radio Access Bearer Service and Core...,The UMTS Bearer Service consists of the Radio ...
1457,question 8114,What does the UE provide to the AS for slice a...,Slice-Maximum Bit Rate,S-NSSAI,NSSRG,S-MBR,Standards overview,option 5: NSAG information,The UE provides the NSAG information and their...
1458,question 8117,What is the interfering signal for the dynamic...,An E-UTRA signal,A CW signal,An AWGN signal,An interfering signal with a positive offset,Standards specifications,option 3: An AWGN signal,The document specifies that the interfering si...
1459,question 8123,What is the purpose of AKMA TLS protocol profi...,To provide secure TLS connections between the ...,To establish a secure tunnel between the netwo...,To generate keys for use in the 5G core,To support non-seamless WLAN offload authentic...,Standards overview,option 1: To provide secure TLS connections be...,AKMA TLS protocol profiles define how to use t...


In [10]:
preamble = \
   'Answer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice'
template = Template('###INSTRUCTION\n$preamble\n\n###QUESTION\n$prompt\n\n###OPTIONS\nA) $a\nB) $b\nC) $c\nD) $d\n###ANSWER\n$ans')

In [11]:
def format_input(df, idx):
    
    prompt = df.loc[idx, 'question']
    a = df.loc[idx, 'option_1']
    b = df.loc[idx, 'option_2']
    c = df.loc[idx, 'option_3']
    d = df.loc[idx, 'option_4']
    ans = df.loc[idx, 'answer']
#     exp = df.loc[idx, 'explanation']
   

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d, ans=ans) #, exp=exp)
    
    return input_text

In [12]:
for i in range(train_df.shape[0]):
    train_df.loc[i, 'text'] = format_input(train_df, i)

In [13]:
train_df['text'][0]

'###INSTRUCTION\nAnswer the following question by selecting the most likely answer choice (A, B, C, or D): please generate only answer choice\n\n###QUESTION\nWhat is the purpose of the Nmfaf_3daDataManagement_Deconfigure service operation? [3GPP Release 18]\n\n###OPTIONS\nA) To configure the MFAF to map data or analytics received by the MFAF to out-bound notification endpoints\nB) To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints\nC) To supply data or analytics from the MFAF to notification endpoints\nD) To fetch data or analytics from the MFAF based on fetch instructions\n###ANSWER\noption 2: To configure the MFAF to stop mapping data or analytics received by the MFAF to out-bound notification endpoints'

In [14]:
train_df.columns

Index(['question_id', 'question', 'option_1', 'option_2', 'option_3',
       'option_4', 'category', 'answer', 'explanation', 'text'],
      dtype='object')

In [15]:
from datasets import Dataset

train_df = Dataset.from_pandas(train_df)
train_df = train_df.remove_columns(['question_id', 'question', 'option_1', 'option_2', 'option_3', 'option_4', 'category', 'answer', 'explanation'])

In [16]:
train_df

Dataset({
    features: ['text'],
    num_rows: 1461
})

# Load 3 datasets

In [17]:
from datasets import load_dataset

dataset1 = load_dataset("Amod/mental_health_counseling_conversations")
dataset2 = load_dataset("mpingale/mental-health-chat-dataset")
dataset3 = load_dataset("heliosbrahma/mental_health_chatbot_dataset")

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

In [18]:
df1= dataset1["train"].data.to_pandas().dropna()
df2= dataset2["train"].data.to_pandas().dropna()
df3= dataset3["train"].data.to_pandas().dropna()

# Preprocess datasets

In [19]:
df1.head()

,Context,Response
0,I'm going through some things with my feelings...,"If everyone thinks you're worthless, then mayb..."
1,I'm going through some things with my feelings...,"Hello, and thank you for your question and see..."
2,I'm going through some things with my feelings...,First thing I'd suggest is getting the sleep y...
3,I'm going through some things with my feelings...,Therapy is essential for those that are feelin...
4,I'm going through some things with my feelings...,I first want to let you know that you are not ...


In [20]:
df2.head()

,questionID,questionTitle,questionText,questionLink,topic,therapistInfo,therapistURL,answerText,upvotes,views,text
0,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Jennifer MolinariHypnotherapist & Licensed Cou...,https://counselchat.com/therapists/jennifer-mo...,It is very common for people to have multiple ...,3,1971,<s>[INST] I have so many issues to address. I ...
1,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Jason Lynch, MS, LMHC, LCAC, ADSIndividual & C...",https://counselchat.com/therapists/jason-lynch...,"I've never heard of someone having ""too many i...",2,386,<s>[INST] I have so many issues to address. I ...
2,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,Shakeeta TorresFaith Based Mental Health Couns...,https://counselchat.com/therapists/shakeeta-to...,Absolutely not. I strongly recommending worki...,2,3071,<s>[INST] I have so many issues to address. I ...
3,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Noorayne ChevalierMA, RP, CCC, CCAC, LLP (Mich...",https://counselchat.com/therapists/noorayne-ch...,Let me start by saying there are never too man...,2,2643,<s>[INST] I have so many issues to address. I ...
4,0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,https://counselchat.com/questions/do-i-have-to...,depression,"Toni Teixeira, LCSWYour road to healing begins...",https://counselchat.com/therapists/toni-teixei...,I just want to acknowledge you for the courage...,1,256,<s>[INST] I have so many issues to address. I ...


In [21]:
# Just rename columns
df2 = df2[["questionText", "answerText"]]
df2.columns = ["question", "answer"]
df2

,question,answer
0,I have so many issues to address. I have a his...,It is very common for people to have multiple ...
1,I have so many issues to address. I have a his...,"I've never heard of someone having ""too many i..."
2,I have so many issues to address. I have a his...,Absolutely not. I strongly recommending worki...
3,I have so many issues to address. I have a his...,Let me start by saying there are never too man...
4,I have so many issues to address. I have a his...,I just want to acknowledge you for the courage...
...,...,...
2770,What are some difficulties that a counselor ca...,Although many clients have the capacity to be ...
2771,What are some difficulties that a counselor ca...,"I usually don't label a client as ""difficult"" ..."
2772,What are some difficulties that a counselor ca...,"Dang right! :)Heh heh, and correct me if I'm ..."
2773,What are some difficulties that a counselor ca...,"Yes, just like some relationships outside of o..."


In [22]:
df3.head()

,text
0,<HUMAN>: What is a panic attack?\n<ASSISTANT>:...
1,<HUMAN>: What are symptoms of panic attack vs....
2,<HUMAN>: What are the types of Mental Illness?...
3,<HUMAN>: What does mental-illness mean?\n<ASSI...
4,<HUMAN>: How can you treat mental illness?\n<A...


In [23]:
# Split single sample into Question & Answer

def split_qa(text):
    # below split returns ["", "question\n", "answer "]
    question, answer = re.split(r" ?<\w+>: ?", text)[1:]
    question = question.strip().replace("\n","")
    answer = answer.strip().replace("\n", "")
    return question, answer
    
df3["text"] = df3["text"].apply(split_qa)
df3[["question", "answer"]] = pd.DataFrame(data=[[q, a] for q, a in df3["text"].values])
df3.drop(columns=["text"], inplace=True)
df3


,question,answer
0,What is a panic attack?,Panic attacks come on suddenly and involve int...
1,What are symptoms of panic attack vs. anxiety ...,Panic attacks and anxiety attacks can share so...
2,What are the types of Mental Illness?,There are many different conditions that are r...
3,What does mental-illness mean?,Mental illness is a group of biological brain ...
4,How can you treat mental illness?,The treatment of mental illness is a multi-fac...
...,...,...
167,Who are you?,Hello there! I am your empathetic Mental Healt...
168,What is the helpline number for suicide preven...,I'm really sorry but I can't provide helpline ...
169,Which is the best mental health hospital in Ne...,"I'm really sorry, but I can't provide accurate..."
170,Who is the best hypnotherapist in London?,I understand that you're looking for a hypnoth...


In [24]:
# Merge datasets
df = pd.concat((df1, df2, df3)).sample(frac=1)
df

,Context,Response,question,answer
1192,I’m trying to make marriage work after a split...,Imagine your best friend just told you that he...,NaN,NaN
74,I’m facing severe depression and anxiety and I...,Anxiety and depression is caused by ineffectiv...,NaN,NaN
2378,NaN,NaN,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...
2570,NaN,NaN,How do you know you have the right therapist f...,It's kind of like an audition. You need to fi...
330,NaN,NaN,How can I get to a place where I can be conten...,It's important to take a look inside and see w...
...,...,...,...,...
3236,Every time I send a message to someone or a gr...,I'm sorry that you are having a difficult time...,NaN,NaN
520,I have a bad habit of thinking and worrying ab...,Thinking about what others think of us is a na...,NaN,NaN
1312,NaN,NaN,I am a survivor of multiple sexual abuse/rape ...,Even if the triggers are having a negative imp...
102,I have been diagnosed with general anxiety and...,"At present, the American Disability Associatio...",NaN,NaN


In [25]:
template = Template('<HUMAN>: $human\n<ASSISTANT>: $assistant')
def format_input(df, idx):
    
    human = df.loc[idx, 'question']
    assistant = df.loc[idx, 'answer']
   

    input_text = template.substitute(
        human=human, assistant=assistant)
    
    return input_text

In [26]:
index = [i for i in range(df.shape[0])]
df_red = df.reset_index(drop=True)
df_index = df_red.reindex(index)
df_index

,Context,Response,question,answer
0,I’m trying to make marriage work after a split...,Imagine your best friend just told you that he...,NaN,NaN
1,I’m facing severe depression and anxiety and I...,Anxiety and depression is caused by ineffectiv...,NaN,NaN
2,NaN,NaN,"I think I have depression, anxiety, bipolar di...",It can be difficult to get counseling if you d...
3,NaN,NaN,How do you know you have the right therapist f...,It's kind of like an audition. You need to fi...
4,NaN,NaN,How can I get to a place where I can be conten...,It's important to take a look inside and see w...
...,...,...,...,...
6291,Every time I send a message to someone or a gr...,I'm sorry that you are having a difficult time...,NaN,NaN
6292,I have a bad habit of thinking and worrying ab...,Thinking about what others think of us is a na...,NaN,NaN
6293,NaN,NaN,I am a survivor of multiple sexual abuse/rape ...,Even if the triggers are having a negative imp...
6294,I have been diagnosed with general anxiety and...,"At present, the American Disability Associatio...",NaN,NaN


In [27]:
for i in range(df_index.shape[0]):
    df_index.loc[i, 'text'] = format_input(df_index, i)

In [28]:
df = df_index

In [29]:
# Remove texts and keep input_ids and labels
df.drop(columns=["question", "answer"], inplace=True)

In [30]:
df['text'][0]

'<HUMAN>: nan\n<ASSISTANT>: nan'

In [31]:
df

,Context,Response,text
0,I’m trying to make marriage work after a split...,Imagine your best friend just told you that he...,<HUMAN>: nan\n<ASSISTANT>: nan
1,I’m facing severe depression and anxiety and I...,Anxiety and depression is caused by ineffectiv...,<HUMAN>: nan\n<ASSISTANT>: nan
2,NaN,NaN,"<HUMAN>: I think I have depression, anxiety, b..."
3,NaN,NaN,<HUMAN>: How do you know you have the right th...
4,NaN,NaN,<HUMAN>: How can I get to a place where I can ...
...,...,...,...
6291,Every time I send a message to someone or a gr...,I'm sorry that you are having a difficult time...,<HUMAN>: nan\n<ASSISTANT>: nan
6292,I have a bad habit of thinking and worrying ab...,Thinking about what others think of us is a na...,<HUMAN>: nan\n<ASSISTANT>: nan
6293,NaN,NaN,<HUMAN>: I am a survivor of multiple sexual ab...
6294,I have been diagnosed with general anxiety and...,"At present, the American Disability Associatio...",<HUMAN>: nan\n<ASSISTANT>: nan


In [32]:
# import gc
# torch.cuda.empty_cache()
# gc.collect()
# torch.cuda.empty_cache()

In [33]:
# Split data into train & test
# train_df, test_df = train_test_split(tokenized_df, test_size=0.15)

# print(f"Training Size: {train_df.shape[0]} samples")
# print(f"Test Size: {test_df.shape[0]} samples")

In [34]:
train_ratio = 0.6
train_size = int(len(df) * train_ratio)

train_dataset = df[:train_size]
eval_dataset = df[train_size:]

In [35]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_dataset)
eval_dataset = Dataset.from_pandas(eval_dataset)
train_dataset = train_dataset.remove_columns('__index_level_0__')
eval_dataset = eval_dataset.remove_columns('__index_level_0__')
# train_dataset = Dataset.from_pandas(df)
# train_dataset = train_dataset.remove_columns('__index_level_0__')

In [36]:
train_dataset

Dataset({
    features: ['Context', 'Response', 'text'],
    num_rows: 3777
})

# Configure PEFT Trainer

In [37]:
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

In [38]:
config = {
    "model1": {
        "train_data": train_df,
        "epochs": 1,
        "path": "./lora-model1"
    },
    "model2": {
        "train_data": train_dataset,
        "epochs": 1,
        "path": "./lora-model2"
    },
}

In [39]:
for i in config['model1']:
    print(i)

train_data
epochs
path


In [40]:
from transformers import AutoTokenizer
from peft import prepare_model_for_kbit_training


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.3")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.3",
    quantization_config=quant_config,
    device_map="auto",
)
model.config.use_cache = False
# model.config.pretraining_tp = 1

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [41]:
# Set pad_token as end-of-sentence token
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [42]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.05
lora_r = 64


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
#     use_rslora=True
)

In [43]:
model = get_peft_model(model, peft_config) 
model.print_trainable_parameters()

trainable params: 27,262,976 || all params: 7,275,286,528 || trainable%: 0.3747


In [44]:
for cfg in config.values():
    output_dir = "./results"
    per_device_train_batch_size = 1
    # per_device_eval_batch_size = 1
    gradient_accumulation_steps = 16
    optim = "paged_adamw_32bit"
    save_steps = 500
    num_train_epochs = cfg['epochs']
    logging_steps = 100
    learning_rate = 6e-5
    max_grad_norm = 0.3
    max_steps = 20
    warmup_ratio = 0.03
    a = ['linear', 'cosine', 'cosine_with_restarts', 'polynomial',\
     'constant', 'constant_with_warmup', 'inverse_sqrt', 'reduce_lr_on_plateau', 'cosine_with_min_lr', 'warmup_stable_decay']
    lr_scheduler_type = a[4]


    training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
#     per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    bf16=False,
    max_grad_norm=max_grad_norm,
    #     max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    )

In [45]:
from trl import SFTTrainer
for cfg in config.values():
    max_seq_length = 512
    # 

    trainer = SFTTrainer(
        model=model,
        train_dataset=cfg['train_data'],
    #     eval_dataset=eval_dataset,
        peft_config=peft_config,
        dataset_text_field="text",
    #     max_seq_length=max_seq_length,
        tokenizer=tokenizer,
        args=training_arguments,
    )

    torch.cuda.empty_cache()
    trainer.train()

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1461 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.17.4
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240712_120802-n56v6w1n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/daniteebee/huggingfa

Step,Training Loss


/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3777 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,1.337100
200,0.977100


# Training

In [46]:
torch.cuda.empty_cache()

In [47]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,0.857700
200,0.843100


TrainOutput(global_step=236, training_loss=0.8590317742299225, metrics={'train_runtime': 6464.7845, 'train_samples_per_second': 0.584, 'train_steps_per_second': 0.037, 'total_flos': 2.16146013259776e+16, 'train_loss': 0.8590317742299225, 'epoch': 0.9997352396081546})

In [ ]:
trainer.model.save_pretrained("./mistral-7B-therapist")
tokenizer.save_pretrained("./mistral-7B-therapist")

# Inference

In [ ]:
def single_inference(question):
    messages = [
        {"role": "system", "content": "Reply to the following inquiry."},
    ]

    messages.append({"role": "user", "content": question})


    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.01,
    #     top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    output = tokenizer.decode(response, skip_special_tokens=True)
    return output

In [ ]:
question = """I feel like I don't exist and my body is not my own, I dont just know what is wrong. what could be wrong with me?"""

answer = single_inference(question)

print(f'INPUT QUESTION:\n{question}')
print(f'\n\nModel Answer:\n{answer}')

In [ ]:
question = """I'm always confused when making decisions, I cannot choose one option, what are the causes of being
indecisive?"""

answer = single_inference(question)

print(f'INPUT QUESTION:\n{question}')
print(f'\n\nModel Answer:\n{answer}')